# Fine-tuning example for MT5 on a custom prompt dataset using `pytorch`

In [29]:
!pip install transformers datasets sentencepiece pandas matplotlib rouge_score nltk torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable


In [30]:
!pip install git+https://github.com/huggingface/transformers.git@master "tokenizers==0.11.2"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git (to revision master) to /tmp/pip-req-build-pd4i316x
  Running command git clone --filter=blob:none -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-pd4i316x
  Resolved https://github.com/huggingface/transformers.git to commit c15bb3fe19b0b6c69a727812cdd3cd5597014667
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [31]:
!sudo apt-get install git-lfs


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
The following packages were automatically installed and are no longer required:
  python-pip-whl python3-wheel
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [32]:
from huggingface_hub import notebook_login

notebook_login()

## Configuration

In [33]:
model_id="philschmid/pt-test"
dataset_id = "philschmid/prompted-germanquad"

## Preprocess dataset

In [34]:
from datasets import load_dataset

ds = load_dataset(dataset_id)

Using custom data configuration philschmid--prompted-germanquad-d89ee1bcbfbf6510
Reusing dataset parquet (/home/ubuntu/.cache/huggingface/datasets/parquet/philschmid--prompted-germanquad-d89ee1bcbfbf6510/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)


  0%|          | 0/1 [00:00<?, ?it/s]

## inspect dataset to get an understanding of `max_input_length` and `max_target_length`

In [35]:
import pandas as pd

In [36]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

https://huggingface.co/t5-small/resolve/main/config.json not found in cache or force_download set to True, downloading to /home/ubuntu/.cache/huggingface/transformers/tmpp0rwz58h


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/config.json in cache at /home/ubuntu/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
creating metadata file for /home/ubuntu/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/spiece.model in cache at /home/ubuntu/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
creating metadata file for /home/ubuntu/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
https://huggingface.co/t5-small/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /home/ubuntu/.cache/huggingface/transformers/tmpogg2r7ms


Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/tokenizer.json in cache at /home/ubuntu/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
creating metadata file for /home/ubuntu/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading file https://huggingface.co/t5-small/resolve/main/spiece.model from cache at /home/ubuntu/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-small/resolve/main/tokenizer.json from cache at /home/ubuntu/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading file https://

In [37]:
df = ds["train"].to_pandas()
df.sample()

,inputs,targets
73976,"Zählen Sie die Zeichen, bis ""Ed Markey"" im fol...",Ed Markey


In [38]:
df["targets"].map(lambda x: len(tokenizer.convert_ids_to_tokens(tokenizer(x).input_ids))).\
    hist()

KeyboardInterrupt: 

In [ ]:
len(len_targets[len_targets > 128])

## tokenize dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

loading file https://huggingface.co/philschmid/pt-test/resolve/main/spiece.model from cache at None
loading file https://huggingface.co/philschmid/pt-test/resolve/main/tokenizer.json from cache at /home/ubuntu/.cache/huggingface/transformers/dbdf24b7ef398595b32b51543c2b6a054eacc4d4bf5802a74444c5441a43547f.5a0fa6cd81d02843cd5508d69831cac79ab709b4ecab561418c227e5c997c9a4
loading file https://huggingface.co/philschmid/pt-test/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/philschmid/pt-test/resolve/main/special_tokens_map.json from cache at /home/ubuntu/.cache/huggingface/transformers/c7bfbc19452e6f193f2a9fed890f6d3ad7a803831422efce0037cc2f59dd10d2.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
loading file https://huggingface.co/philschmid/pt-test/resolve/main/tokenizer_config.json from cache at /home/ubuntu/.cache/huggingface/transformers/7e6a849165a1a834bf8bcf017c656fc18dc40d52541d56f8fd28b5eebd852509.dd07407edab597040f343f0e281e

In [24]:
ds

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 164664
    })
})

In [39]:
ds["train"][0]

{'inputs': 'Ich weiß, dass die Antwort auf "Von welchem Gesetzt stammt das Amerikanische ab? " irgendwo im folgenden Textausschnitt. Können Sie mir sagen, an welchem Zeichen die Antwort beginnt?\n\nRecht_der_Vereinigten_Staaten\n\n=== Amerikanisches Common Law ===\nObwohl die Vereinigten Staaten wie auch viele Staaten des Commonwealth Erben des britischen Common Laws sind, setzt sich das amerikanische Recht bedeutend davon ab. Dies rührt größtenteils von dem langen Zeitraum her, in dem sich das amerikanische Recht unabhängig vom Britischen entwickelt hat. Entsprechend schauen die Gerichte in den Vereinigten Staaten bei der Analyse von eventuell zutreffenden britischen Rechtsprinzipien im Common Law gewöhnlich nur bis ins frühe 19. Jahrhundert.\nWährend es in den Commonwealth-Staaten üblich ist, dass Gerichte sich Entscheidungen und Prinzipien aus anderen Commonwealth-Staaten importieren, ist das in der amerikanischen Rechtsprechung selten. Ausnahmen bestehen hier nur, wenn sich überhau

In [26]:
max_input_length = 512
max_target_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=max_input_length, truncation=True
    )
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["targets"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [27]:
tokenized_datasets = ds["train"].map(preprocess_function, batched=True)


  0%|          | 0/165 [00:00<?, ?ba/s]

In [49]:
# test size will be 15% of train dataset
test_size=.15
seed=33

processed_dataset = tokenized_datasets.shuffle(seed=seed).train_test_split(test_size=test_size)
# processed_dataset = tokenized_datasets.shuffle(seed=seed).select(range(2000)).train_test_split(test_size=test_size)
processed_dataset

Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/parquet/philschmid--prompted-germanquad-d89ee1bcbfbf6510/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-f712ad42dfd3623f.arrow


DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 139964
    })
    test: Dataset({
        features: ['inputs', 'targets', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 24700
    })
})

In [50]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 139964
    })
    test: Dataset({
        features: ['inputs', 'targets', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 24700
    })
})

# Train the model

In [40]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

loading configuration file https://huggingface.co/philschmid/pt-test/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/1ff5bf4d63090c63b6afae24547496ef77d1af6f0eb571d934c0e5878682748a.87273989f1f4b3e294212e68cb68ac618c9d28aac6164324c02784fad4773733
Model config T5Config {
  "_name_or_path": "philschmid/pt-test",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.16.0.dev0",
  "use_cache": true,
  "vocab_size": 32103
}

loading weights file ht

In [41]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [42]:
from transformers import Seq2SeqTrainingArguments
from huggingface_hub import HfFolder

per_device_train_batch_size = 4
per_device_eval_batch_size = 4
num_train_epochs = 3
learning_rate = 5.6e-5
# Show the training loss with every epoch
logging_steps = len(processed_dataset["train"]) // per_device_train_batch_size
model_name = model_id.split("/")[-1]
output_dir=f"{model_name}-prompted-germanquad"
hub_token = HfFolder.get_token() # or your token directly "hf_xxx"
hub_model_id = f'{model_id.split("/")[1]}-{dataset_id}-v1'

args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    learning_rate=learning_rate,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    predict_with_generate=True,
    logging_steps=logging_steps,
    load_best_model_at_end=True,
    metric_for_best_model="rougeLsum",
    # push_to_hub=True,
    # hub_model_id=hub_model_id,
    # hub_token=hub_token,   
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
import numpy as np
from nltk.tokenize import sent_tokenize
from datasets import load_metric

rouge_score = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [45]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [46]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: targets, inputs.
/home/ubuntu/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1700
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1275


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.446400,1.955729,13.425900,7.238800,13.047600,13.082200
2,1.669300,1.866318,21.101900,14.553900,20.666600,20.843100
3,1.409400,1.850856,21.493800,14.643300,21.054900,21.251200


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: targets, inputs.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 4
Saving model checkpoint to pt-test-prompted-germanquad/checkpoint-425
Configuration saved in pt-test-prompted-germanquad/checkpoint-425/config.json
Model weights saved in pt-test-prompted-germanquad/checkpoint-425/pytorch_model.bin
tokenizer config file saved in pt-test-prompted-germanquad/checkpoint-425/tokenizer_config.json
Special tokens file saved in pt-test-prompted-germanquad/checkpoint-425/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: targets, inputs.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 4
Saving model checkpoint to pt-test-prompted-germanquad/checkpoint-850
Configuration saved in pt-test-prompted-german

TrainOutput(global_step=1275, training_loss=1.8416911285998774, metrics={'train_runtime': 480.0445, 'train_samples_per_second': 10.624, 'train_steps_per_second': 2.656, 'total_flos': 3242426575804416.0, 'train_loss': 1.8416911285998774, 'epoch': 3.0})

In [ ]:
trainer.evaluate()


In [ ]:
trainer.push_to_hub(commit_message="Training complete", tags="text2text")
